In [1]:
import sympy as sym
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
def Christoffel_Sym (g, x): ### Takes in metric and array-like of sympy symbols
    n = g.shape[0]
    gamma = sym.MutableDenseNDimArray([0]*n**3, (n,n,n))
    for i in range(n):
        for j in range(n):
            for k in range(n):
                gamma[i,j,k] = 1/2 * (g**-1)[i,i] * (sym.diff(g[k,i],x[j])+sym.diff(g[j,i],x[k])-sym.diff(g[j,k],x[i]))
    return gamma

In [3]:
t, r, theta, phi = sym.symbols('t, r, theta, phi')
rs, c , rq= sym.symbols('r_s, c, rq')

In [4]:
# Euclidean
g1 = sym.Matrix([[1,0],
                [0,1]])
# Polar
g2 = sym.Matrix([[1,0],
                [0,r**2]])
# Spherical
g3 = sym.Matrix([[1,0,0],
                [0,r**2,0],
                [0,0,(r*sym.sin(theta))**2]])
# S^2
g4 = sym.Matrix([[r**2,0],
                [0,(r*sym.sin(theta))**2]])
# Schwarzchild
g5 = sym.Matrix([[-c**2*(1-rs/r),0,0,0],
                 [0,1/(1-rs/r),0,0],
                 [0,0,r**2,0],
                 [0,0,0,r**2*sym.sin(theta)**2]])
# Reissner-Nordstrom
g6 = sym.Matrix([[-(1-rs/r+rq**2/r**2),0,0,0],
                 [0,1/(1-rs/r+rq**2/r**2),0,0],
                 [0,0,r**2,0],
                 [0,0,0,r**2*sym.sin(theta)**2]])

In [5]:
x = [r,theta]
Christoffel_Sym(g2,x)

[[[0, 0], [0, -1.0*r]], [[0, 1.0/r], [1.0/r, 0]]]

In [6]:
x = [r,theta,phi]
Christoffel_Sym(g3,x)

[[[0, 0, 0], [0, -1.0*r, 0], [0, 0, -1.0*r*sin(theta)**2]], [[0, 1.0/r, 0], [1.0/r, 0, 0], [0, 0, -1.0*sin(theta)*cos(theta)]], [[0, 0, 1.0/r], [0, 0, 1.0*cos(theta)/sin(theta)], [1.0/r, 1.0*cos(theta)/sin(theta), 0]]]

In [7]:
x = [theta,phi]
g4
Christoffel_Sym(g4,x)

Matrix([
[r**2,                  0],
[   0, r**2*sin(theta)**2]])

[[[0, 0], [0, -1.0*sin(theta)*cos(theta)]], [[0, 1.0*cos(theta)/sin(theta)], [1.0*cos(theta)/sin(theta), 0]]]

In [8]:
def reimann_tensor(gamma,x):
    n = gamma.shape[0]
    R = sym.MutableDenseNDimArray([0]*n**4, (n,n,n,n))

    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    R[i,j,k,l] = -sym.diff(gamma[i,j,k],x[l])+sym.diff(gamma[i,j,l],x[k])
                    for m in range(n):

                        R[i,j,k,l] += gamma[i,m,k]*gamma[m,j,l] - gamma[i,m,l]*gamma[m,j,k]
    return sym.simplify(R)

In [9]:
def ricci_tensor (R):
    n = R.shape[0]
    ricci_t = sym.MutableDenseNDimArray([0]*n**2, (n,n))
    
    for i in range(n):
        for j in range(n):
            for k in range(n):
                ricci_t[i,j] += R[k,i,j,k]
    return sym.simplify(ricci_t)

In [10]:
def ricci_scalar(R,g):
    n = R.shape[0]
    ricci_s = 0
    for i in range(n):
        for j in range(n):
            ricci_s += (g**-1)[i,j]*R[j,i]
    return sym.simplify(ricci_s)

In [11]:
x = [t,r,theta,phi]
gamma = Christoffel_Sym(g5,x)
R = reimann_tensor(gamma,x)
x
gamma
R

[t, r, theta, phi]

[[[0, 0.5*r_s/(r**2*(1 - r_s/r)), 0, 0], [0.5*r_s/(r**2*(1 - r_s/r)), 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[c**2*r_s*(0.5 - 0.5*r_s/r)/r**2, 0, 0, 0], [0, -r_s*(0.5 - 0.5*r_s/r)/(r**2*(1 - r_s/r)**2), 0, 0], [0, 0, -2*r*(0.5 - 0.5*r_s/r), 0], [0, 0, 0, -2*r*(0.5 - 0.5*r_s/r)*sin(theta)**2]], [[0, 0, 0, 0], [0, 0, 1.0/r, 0], [0, 1.0/r, 0, 0], [0, 0, 0, -1.0*sin(theta)*cos(theta)]], [[0, 0, 0, 0], [0, 0, 0, 1.0/r], [0, 0, 0, 1.0*cos(theta)/sin(theta)], [0, 1.0/r, 1.0*cos(theta)/sin(theta), 0]]]

[[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1.0*r_s/(r**2*(r - r_s)), 0, 0], [-1.0*r_s/(r**2*(r - r_s)), 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, -0.5*r_s/r, 0], [0, 0, 0, 0], [0.5*r_s/r, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, -0.5*r_s*sin(theta)**2/r], [0, 0, 0, 0], [0, 0, 0, 0], [0.5*r_s*sin(theta)**2/r, 0, 0, 0]]], [[[0, 1.0*c**2*r_s*(r - r_s)/r**4, 0, 0], [1.0*c**2*r_s*(-r + r_s)/r**4, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, -0.5*r_s/r, 0], [0, 0.5*r_s/r, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, -0.5*r_s*sin(theta)**2/r], [0, 0, 0, 0], [0, 0.5*r_s*sin(theta)**2/r, 0, 0]]], [[[0, 0, 0.5*c**2*r_s*(-r + r_s)/r**4, 0], [0, 0, 0, 0], [0.5*c**2*r_s*(r - r_s)/r**4, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0.5*r_s/(r**2*(r - r_s)), 0], [0, -0.5*r_s/(r**2*(r - r_s)), 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0,

In [12]:
ricci_t = ricci_tensor(R)
ricci_t

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

In [13]:
ricci_s = ricci_scalar(ricci_t,g5)
ricci_s

0

In [14]:
x = [t,r,theta,phi]
gamma = Christoffel_Sym(g6,x)
R = reimann_tensor(gamma,x)
x
gamma
R

[t, r, theta, phi]

[[[0, 0.5*(-r_s/r**2 + 2*rq**2/r**3)/(-1 + r_s/r - rq**2/r**2), 0, 0], [0.5*(-r_s/r**2 + 2*rq**2/r**3)/(-1 + r_s/r - rq**2/r**2), 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[(r_s/r**2 - 2*rq**2/r**3)*(0.5 - 0.5*r_s/r + 0.5*rq**2/r**2), 0, 0, 0], [0, (-r_s/r**2 + 2*rq**2/r**3)*(0.5 - 0.5*r_s/r + 0.5*rq**2/r**2)/(1 - r_s/r + rq**2/r**2)**2, 0, 0], [0, 0, -2*r*(0.5 - 0.5*r_s/r + 0.5*rq**2/r**2), 0], [0, 0, 0, -2*r*(0.5 - 0.5*r_s/r + 0.5*rq**2/r**2)*sin(theta)**2]], [[0, 0, 0, 0], [0, 0, 1.0/r, 0], [0, 1.0/r, 0, 0], [0, 0, 0, -1.0*sin(theta)*cos(theta)]], [[0, 0, 0, 0], [0, 0, 0, 1.0/r], [0, 0, 0, 1.0*cos(theta)/sin(theta)], [0, 1.0/r, 1.0*cos(theta)/sin(theta), 0]]]

[[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1.0*(1.0*r*r_s - 3.0*rq**2)/(r**2*(r**2 - r*r_s + rq**2)), 0, 0], [(-1.0*r*r_s + 3.0*rq**2)/(r**2*(r**2 - r*r_s + rq**2)), 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1.0*(-0.5*r*r_s + 1.0*rq**2)/r**2, 0], [0, 0, 0, 0], [0.5*(r*r_s - 2*rq**2)/r**2, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, (-0.5*r*r_s + 1.0*rq**2)*sin(theta)**2/r**2], [0, 0, 0, 0], [0, 0, 0, 0], [0.5*(r*r_s - 2*rq**2)*sin(theta)**2/r**2, 0, 0, 0]]], [[[0, 1.0*(1.0*r**3*r_s - 1.0*r**2*r_s**2 - 3.0*r**2*rq**2 + 4.0*r*r_s*rq**2 - 3.0*rq**4)/r**6, 0, 0], [1.0*(-1.0*r**3*r_s + 1.0*r**2*r_s**2 + 3.0*r**2*rq**2 - 4.0*r*r_s*rq**2 + 3.0*rq**4)/r**6, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, (-0.5*r*r_s + 1.0*rq**2)/r**2, 0], [0, (0.5*r*r_s - 1.0*rq**2)/r**2, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, (-0.5*r*r_s + 1.0*rq**2)*sin(theta)**2/r**2], [0, 0, 0, 0], [0, (0.5*r*r_s - 1.0*rq**2)*

In [15]:
ricci_t = ricci_tensor(R)
ricci_t

[[1.0*rq**2*(-r**2 + r*r_s - rq**2)/r**6, 0, 0, 0], [0, 1.0*rq**2/(r**2*(r**2 - r*r_s + rq**2)), 0, 0], [0, 0, -1.0*rq**2/r**2, 0], [0, 0, 0, -1.0*rq**2*sin(theta)**2/r**2]]

In [16]:
ricci_s = ricci_scalar(ricci_t,g5)
ricci_s

rq**2*(1.0*c**2*r**2*(r - r_s)**2 - 2.0*c**2*r*(r - r_s)*(r**2 - r*r_s + rq**2) + 1.0*(r**2 - r*r_s + rq**2)**2)/(c**2*r**5*(r - r_s)*(r**2 - r*r_s + rq**2))

In [24]:
g5.det()

-c**2*r**4*sin(theta)**2